In [6]:
 #!pip install rioxarray "xarray[complete]" esmpy  numpy shapely cf_xarray sparse numba


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
ERROR: Could not find a version that satisfies the requirement esmpy (from versions: none)
ERROR: No matching distribution found for esmpy


In [7]:
#!conda install -c conda-forge esmpy xarray numpy shapely cf_xarray sparse numba

Retrieving notices: ...working... done
Solving environment: | ^C
failed with initial frozen solve. Retrying with flexible solve.


In [13]:
import rioxarray as rx
import pandas as pd
import boto3
import xarray as xr
import glob
import shutil
import os
import xesmf as xe
import numpy as np

In [7]:
df = filesToProcess(s3Files=s3Files,
                    year=1980)
s3 = boto3.client("s3")
outputPath = "data"
originaExtension = "nc"
for index, row in df.iterrows():
    _file_name = row['file_name']
    _s3_path = row['s3_path']
    downloadFile = outputPath+"/"+_file_name+"."+originaExtension
    downloadData(bucketName=bucketName
            , s3Path=_s3_path
            , outputName=downloadFile
            , s3=s3
                )
    break

In [9]:
geoFile= xr.open_dataset("data/Temperature-Air-2m-Max-24h_C3S-glob-agric_AgERA5_19800101_final-v1.nc")

#geoFileC = geoFile.apply(convertKelvinCelsious)
#variable = list(geoFileC.keys())[0]

In [16]:
grid5degrees =  xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90, 90, 0.5)),
        "lon": (["lon"], np.arange(-180, 180, 0.5)),
    }
)


In [17]:
grid5degrees

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 720)
Coordinates:
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.0 88.5 89.0 89.5
  * lon      (lon) float64 -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
Data variables:
    *empty*

In [25]:
from datetime import datetime
firsttime = datetime.now()
regridder = xe.Regridder(geoFile, grid5degrees, "bilinear")
lasttime = datetime.now()
print(lasttime-firsttime)

0:02:09.232619


In [26]:
regridder.to_netcdf()

'bilinear_1801x3600_360x720.nc'

In [27]:
rg= xr.open_dataset("bilinear_1801x3600_360x720.nc")

In [28]:
rg

<xarray.Dataset>
Dimensions:  (n_s: 1036800)
Dimensions without coordinates: n_s
Data variables:
    S        (n_s) float64 ...
    col      (n_s) int32 ...
    row      (n_s) int32 ...

In [20]:
ds_out = regridder(geoFile)

In [22]:
ds_out.to_netcdf("data/chagin_degrees.nc")

In [ ]:
https://xesmf.readthedocs.io/en/latest/notebooks/Reuse_regridder.html